In [ ]:
#This script takes every simulation with all points and calculates rmse and mae in each case

In [7]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
import matplotlib.gridspec as gridspec
import ast
import sys
sys.path.append('machine-scientist/')
sys.path.append('machine-scientist/Prior/')
from mcmc import *
from parallel import *
from fit_prior import read_prior_par
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import mean_absolute_error

In [8]:
def clean_index(dataframe):
    dataframe.set_index('Unnamed: 0', inplace=True)
    dataframe.index.name = None
    dataframe= dataframe.reset_index(drop=True)
    return dataframe

def add_bms_pred(dataframe, bms_trace, number_param):
    VARS = ['x1',]
    x = dn[[c for c in VARS]].copy()
    y=dataframe.noise

    if number_param==10:
        prior_par = read_prior_par('machine-scientist/Prior/final_prior_param_sq.named_equations.nv1.np10.2017-10-18 18:07:35.089658.dat')
    elif number_param==20:
        prior_par = read_prior_par('machine-scientist/Prior/final_prior_param_sq.named_equations.nv1.np20.maxs200.2024-05-10 162907.551306.dat')

    #mdl model
    minrow = bms_trace[bms_trace.H == min(bms_trace.H)].iloc[0]
    minH, minexpr, minparvals = minrow.H, minrow.expr, ast.literal_eval(minrow.parvals)

    t = Tree(
        variables=list(x.columns),
        parameters=['a%d' % i for i in range(number_param)],
        x=x, y=y,
        prior_par=prior_par,
        max_size=200,
        from_string=minexpr,
    )

    t.set_par_values(deepcopy(minparvals))

    dplot = deepcopy(dn)
    dplot['ybms'] = t.predict(x)

    return dplot
    

In [18]:
#Read NN and BMS data
functions=['leaky_ReLU', 'tanh'] #tanh, leaky_ReLU
realizations=2
N=9
sigmas=[0.0, 0.02, 0.04,0.06, 0.08, 0.10, 0.12, 0.14, 0.16, 0.18, 0.20]
resolution='0.5x' #0.5x, 1x, 2x

if resolution=='1x':
    train_size=60
elif resolution=='0.5x':
    train_size=30
elif resolution=='2x':
    train_size=120

runid=0
NPAR=10 #10, 20
steps=50000


rmse_nn_train=[];rmse_nn_test=[]
rmse_mdl_train=[];rmse_mdl_test=[]

mae_nn_train=[];mae_nn_test=[]
mae_mdl_train=[];mae_mdl_test=[]

n_index=[];r_index=[];sigma_index=[];function_index=[]

#Put mae and rmse of each simulation (on nn and bms) in a dataframe
for function in functions:

    for sigma in sigmas:

        for r in range(realizations+1):
            
            file_model='NN_no_overfit_' + function + '_sigma_' + str(sigma) + '_r_' + str(r) + '.csv'
            model_d='../data/' + resolution + '_resolution/trained_nns/' + file_model
            d=pd.read_csv(model_d)

            for n in range(N+1):
                #print(n)
                n_index.append(n);r_index.append(r);sigma_index.append(sigma);function_index.append(function)
            
                dn=d[d['rep']==n]
                dn=clean_index(dn)

                #Read BMS data
                if resolution=='1x':
                    filename='BMS_'+function+'_n_'+str(n)+'_sigma_'+str(sigma)+ '_r_' + str(r) + '_trace_'+str(steps)+'_prior_'+str(NPAR)+ '.csv'
                elif resolution=='0.5x':
                    filename='BMS_'+function+'_n_'+str(n)+'_sigma_'+str(sigma)+ '_r_' + str(r) + '_res_0.1_trace_'+str(steps)+'_prior_'+str(NPAR)+ '.csv'
                elif resolution=='2x':
                    filename='BMS_'+function+'_n_'+str(n)+'_sigma_'+str(sigma)+ '_r_' + str(r) + '_res_0.025_trace_'+str(steps)+'_prior_'+str(NPAR)+ '.csv'

                print(function, sigma, n, r)
                trace=pd.read_csv('../data/MSTraces/' + resolution + '_resolution/' + filename, sep=';', header=None, names=['t', 'H', 'expr', 'parvals', 'kk1', 'kk2','kk3'])
                dplot=add_bms_pred(dn, trace, NPAR)

                #Errors
            
                #nns
                rmse_nn_train_i=root_mean_squared_error(dplot.loc[:train_size-1]['ymodel'],dplot.loc[:train_size -1]['y'])
                rmse_nn_train.append(rmse_nn_train_i)
            
                rmse_nn_test_i=root_mean_squared_error(dplot.loc[train_size-1:]['ymodel'],dplot.loc[train_size -1:]['y'])
                rmse_nn_test.append(rmse_nn_test_i)

                mae_nn_train_i=mean_absolute_error(dplot.loc[:train_size-1]['ymodel'],dplot.loc[:train_size -1]['y'])
                mae_nn_train.append(mae_nn_train_i)
            
                mae_nn_test_i=mean_absolute_error(dplot.loc[train_size-1:]['ymodel'],dplot.loc[train_size -1:]['y'])
                mae_nn_test.append(mae_nn_test_i)


                try:
                    rmse_mdl_train_i=root_mean_squared_error(dplot.loc[:train_size-1]['ybms'],dn.loc[:train_size-1]['y'])
                except ValueError:
                    rmse_mdl_train_i=np.inf
                rmse_mdl_train.append(rmse_mdl_train_i)

                try:
                    rmse_mdl_test_i=root_mean_squared_error(dplot.loc[train_size-1:]['ybms'],dn.loc[train_size-1:]['y'])
                except (ValueError, RuntimeWarning) as e:
                    rmse_mdl_test_i=np.inf
                
                rmse_mdl_test.append(rmse_mdl_test_i)

                try:
                    mae_mdl_train_i=mean_absolute_error(dplot.loc[:train_size-1]['ybms'],dplot.loc[:train_size -1]['y'])
                except ValueError:
                    mae_mdl_train_i=np.inf
                mae_mdl_train.append(mae_mdl_train_i)

                try:
                    mae_mdl_test_i=mean_absolute_error(dplot.loc[train_size-1:]['ybms'],dplot.loc[train_size -1:]['y'])
                except ValueError:
                    mae_mdl_test_i=np.inf
                
                mae_mdl_test.append(mae_mdl_test_i)

errors_df=pd.DataFrame({'sigma':sigma_index, 'function':function_index, 'mae_nn_train':mae_nn_train, 'mae_nn_test':mae_nn_test, 'mae_mdl_train':mae_mdl_train, 
                        'mae_mdl_test':mae_mdl_test, 'rmse_nn_train':rmse_nn_train, 'rmse_nn_test': rmse_nn_test, 
                        'rmse_mdl_train':rmse_mdl_train, 'rmse_mdl_test': rmse_mdl_test, 'n':n_index, 'r': r_index})
errors_df.to_csv('../data/all_errors_' + resolution + '.csv')
display(errors_df)

leaky_ReLU 0.0 0 0
leaky_ReLU 0.0 1 0
leaky_ReLU 0.0 2 0
leaky_ReLU 0.0 3 0
leaky_ReLU 0.0 4 0
leaky_ReLU 0.0 5 0
leaky_ReLU 0.0 6 0
leaky_ReLU 0.0 7 0
leaky_ReLU 0.0 8 0
leaky_ReLU 0.0 9 0
leaky_ReLU 0.0 0 1
leaky_ReLU 0.0 1 1
leaky_ReLU 0.0 2 1
leaky_ReLU 0.0 3 1
leaky_ReLU 0.0 4 1
leaky_ReLU 0.0 5 1
leaky_ReLU 0.0 6 1
leaky_ReLU 0.0 7 1
leaky_ReLU 0.0 8 1
leaky_ReLU 0.0 9 1
leaky_ReLU 0.0 0 2
leaky_ReLU 0.0 1 2
leaky_ReLU 0.0 2 2
leaky_ReLU 0.0 3 2
leaky_ReLU 0.0 4 2
leaky_ReLU 0.0 5 2
leaky_ReLU 0.0 6 2
leaky_ReLU 0.0 7 2
leaky_ReLU 0.0 8 2
leaky_ReLU 0.0 9 2
leaky_ReLU 0.02 0 0
leaky_ReLU 0.02 1 0
leaky_ReLU 0.02 2 0
leaky_ReLU 0.02 3 0
leaky_ReLU 0.02 4 0
leaky_ReLU 0.02 5 0
leaky_ReLU 0.02 6 0
leaky_ReLU 0.02 7 0
leaky_ReLU 0.02 8 0
leaky_ReLU 0.02 9 0
leaky_ReLU 0.02 0 1
leaky_ReLU 0.02 1 1
leaky_ReLU 0.02 2 1
leaky_ReLU 0.02 3 1
leaky_ReLU 0.02 4 1
leaky_ReLU 0.02 5 1
leaky_ReLU 0.02 6 1
leaky_ReLU 0.02 7 1
leaky_ReLU 0.02 8 1
leaky_ReLU 0.02 9 1
leaky_ReLU 0.02 0 2
leaky_ReLU

,sigma,function,mae_nn_train,mae_nn_test,mae_mdl_train,mae_mdl_test,rmse_nn_train,rmse_nn_test,rmse_mdl_train,rmse_mdl_test,n,r
0,0.0,leaky_ReLU,0.016896,0.029447,0.001032,0.063266,0.022505,0.036252,0.001275,0.082951,0,0
1,0.0,leaky_ReLU,0.016883,0.231939,0.023344,0.269050,0.021710,0.254277,0.028029,0.296079,1,0
2,0.0,leaky_ReLU,0.018241,0.103689,0.014257,0.086586,0.022336,0.135278,0.016435,0.110833,2,0
3,0.0,leaky_ReLU,0.079068,0.335331,0.004799,0.203640,0.094522,0.347979,0.006336,0.246610,3,0
4,0.0,leaky_ReLU,0.010155,0.044933,0.003880,0.506176,0.012755,0.050680,0.004796,0.726888,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...
655,0.2,tanh,0.104493,0.377508,0.060303,0.831309,0.135553,0.394259,0.106253,0.890840,5,2
656,0.2,tanh,0.118410,0.133384,0.044108,0.263869,0.156538,0.136299,0.066985,0.266259,6,2
657,0.2,tanh,0.157235,0.070630,0.094496,0.012186,0.203894,0.077455,0.113941,0.017034,7,2
658,0.2,tanh,0.151484,0.143675,0.163269,0.748055,0.192902,0.170953,0.224000,0.750894,8,2
